# **Apply 12-Term Cal from SOLT Calibration**

In [ ]:
import numpy as np
from pathlib import Path
import skrf as rf
from skrf.calibration import OnePort
from cycler import cycler

import matplotlib.pyplot as plt
rf.stylely

n_freq=101
freq = rf.Frequency(8, 12, n_freq, 'ghz')

plt.rcParams['axes.prop_cycle'] = cycler(color=['b', 'r', 'g', 'y'])

### Define ideal cals corresponding to Agilent 85521A mechanical cal kit

#### *Ideal Thru*

In [ ]:
base_ntwk = rf.Freespace(freq, z0=50+0j)

THRU_85521A = base_ntwk.line(115.881,unit ='ps')
THRU_85521A.z0 = (50+0j) * np.ones((n_freq, 4)) 

fig, ax = plt.subplots()
THRU_85521A.plot_s_re(axis=ax, linewidth=2)
THRU_85521A.plot_s_im(axis=ax, linestyle='--', linewidth=2)
ax.set_title('Ideal THRU')

#### *Ideal Open*

In [ ]:
OPEN_S11_85521A = base_ntwk.delay_load(1, 31.832, unit = 'ps')
OPEN_S22_85521A = OPEN_S11_85521A 

OPEN_85521A = rf.two_port_reflect(OPEN_S11_85521A, OPEN_S22_85521A)
OPEN_85521A.z0 = (50+0j) * np.ones((n_freq, 4)) 

fig, ax = plt.subplots()
OPEN_85521A.plot_s_re(axis =ax, linewidth= 2)
OPEN_85521A.plot_s_im(axis =ax, linestyle='--', linewidth= 2)
ax.set_title('Ideal OPEN')

#### *Ideal Short*

In [ ]:
SHORT_S11_85521A = base_ntwk.delay_load(-1, 30.581, unit = 'ps')

SHORT_S22_85521A = SHORT_S11_85521A

SHORT_85521A = rf.two_port_reflect(SHORT_S11_85521A,SHORT_S22_85521A)
SHORT_85521A.z0 = (50+0j) * np.ones((n_freq, 4)) 

fig, ax = plt.subplots()
SHORT_85521A.plot_s_re(axis=ax, linewidth=2)
SHORT_85521A.plot_s_im(axis=ax, linestyle='--', linewidth=2)

ax.set_title('Ideal SHORT')

#### *Ideal Load*

In [ ]:
LOAD_S11_85521A = base_ntwk.delay_load(0,0)

LOAD_85521A = rf.two_port_reflect(LOAD_S11_85521A)
LOAD_85521A.z0 = (50+0j) * np.ones((n_freq, 4)) 

fig, ax = plt.subplots()
LOAD_85521A.plot_s_re(axis=ax, linewidth=2)
LOAD_85521A.plot_s_im(axis=ax, linestyle='--', linewidth=2)

ax.set_title('Ideal LOAD')

In [ ]:
#list of Network types, holding 'ideal' responses ORDER MUST BE SAME AS THOSE ENTERED BELOW IN MEASUREMENTS
my_ideals = [\
        SHORT_85521A,
        OPEN_85521A,
        LOAD_85521A,
        THRU_85521A,
        ]

### Load measured response from mechanical cal (85521A)

In [ ]:
measurement_root_folder = Path('/Users/Zaber/Documents/data/scikit_measurements/Mike_Metasurface_3/')
cal_folder_name = 'SOLT2021-04-08'

meas_folder = measurement_root_folder / cal_folder_name

SHORT_S11 = np.load(meas_folder / 'SHORT_S11.ntwk', allow_pickle = True)
SHORT_S22 = np.load(meas_folder / 'SHORT_S22.ntwk', allow_pickle = True)
OPEN_S11 = np.load(meas_folder / 'OPEN_S11.ntwk', allow_pickle = True)
OPEN_S22 = np.load( meas_folder / 'OPEN_S22.ntwk', allow_pickle = True)
LOAD_S11 = np.load(meas_folder / 'LOAD_S11.ntwk', allow_pickle = True)
LOAD_S22 = np.load(meas_folder / 'LOAD_S22.ntwk', allow_pickle = True)
THRU_S11 = np.load(meas_folder / 'THRU_S11.ntwk', allow_pickle = True)
THRU_S22 = np.load(meas_folder / 'THRU_S22.ntwk', allow_pickle = True)
THRU_S21 = np.load(meas_folder / 'THRU_S21.ntwk', allow_pickle = True)
THRU_S12 = np.load(meas_folder / 'THRU_S12.ntwk', allow_pickle = True)

SHORT = rf.two_port_reflect(SHORT_S11,SHORT_S22)
SHORT.z0 = (50+0j) * np.ones((n_freq, 4)) 

OPEN = rf.two_port_reflect(OPEN_S11,OPEN_S22)
OPEN.z0 = (50+0j) * np.ones((n_freq, 4)) 
 
LOAD = rf.two_port_reflect(LOAD_S11,LOAD_S22)
LOAD.z0 = (50+0j) * np.ones((n_freq, 4)) 

THRU = rf.two_port_reflect(THRU_S11, THRU_S22)
THRU.s[:,0,1] = THRU_S12.s[:,0,0]
THRU.s[:,1,0] = THRU_S21.s[:,0,0]
THRU.z0 = (50+0j) * np.ones((n_freq, 4)) 
    
my_measured = [\
               SHORT,
               OPEN,
               LOAD,
               THRU
        ]

### Create a calibration instance

In [ ]:
cal = rf.TwelveTerm(\
        ideals = my_ideals,
        measured = my_measured,
        )

### Run SOLT calibration algorithm

In [ ]:
cal.run()

### Test on THRU measurement

In [ ]:
dut = THRU
dut.name = 'test'
dut_caled = cal.apply_cal(dut)
dut_caled.name =  dut.name + 'corrected'

# plot results
dut_caled.frequency.unit = 'ghz'

dut_caled.plot_s_deg()

### Save result

In [ ]:
dut_caled.write_touchstone()